# Database examples

## Premise

Let's assume we want to upload a newly created isotherm in the internal database. This isotherm
is measured on the novel adsorbent *Carbon X1*, with nitrogen at 77 K. The sample itself is 
synthesised locally, by Dr. Alice and has then undergone a chemical activation process. The experiment is performed on a homemade apparatus by Dr. Bob.

## Imports

First we need to do the required imports

In [1]:
import pygaps

Selected 0 samples
Selected 23 adsorbates


## Adsorbate

The internal database already contains nitrogen as an adsorbate therefore, there's no need to
worry about the adsorbate for now.

## Sample

Since no samples are present in the internal database, we must first upload the sample object.
We create a Sample class with the required values and try to upload it.

In [2]:
novel_sample = pygaps.Sample({
            'name' : 'Carbon',
            'batch' : 'X1',

            'contact' : 'Alice',
            'source' : 'local',
            'type' : 'novel',
    
            'parameters' : {
                'treatment' : 'chemical'
            }
        })

try:
    pygaps.db_upload_sample(pygaps.DATABASE, novel_sample)
except Exception as e:
    print(e)

Error on sample: 
 Carbon X1 
 FOREIGN KEY constraint failed



We get a foreign key error, since the constraints do not exist in the database yet.
Before we can upload the sample, we need to have the values of the foreign keys in place:

- the contact, as Alice
- the source, as local
- the type of adsorbent, as novel
- the sample property we have added

We can do this by passing to the respective functions dictionaries with the required values.

In [3]:
pygaps.db_upload_contact(pygaps.DATABASE, 
                         {
                             'nick' : 'Alice',
                             'name' : 'Dr. Alice'
                         })

Contact uploaded Alice


In [4]:
pygaps.db_upload_source(pygaps.DATABASE, 
                         {
                             'nick' : 'local',
                             'name' : 'our own lab'
                         })

Source uploaded local


In [5]:
pygaps.db_upload_sample_type(pygaps.DATABASE, 
                             {
                                 'type' : 'novel',
                                 'name' : 'a novel sample which was developed in our lab'
                             })

Sample type uploaded {'type': 'novel', 'name': 'a novel sample which was developed in our lab'}


In [6]:
pygaps.db_upload_sample_property_type(pygaps.DATABASE, 
                             {
                                 'type' : 'chemical',
                             })

Property type uploaded {'type': 'chemical'}


Now that all of the constraints are in place let's try the sample upload again.

In [7]:
pygaps.db_upload_sample(pygaps.DATABASE, novel_sample)

Sample uploaded Carbon X1


## Isotherm

Now it's time to upload the isotherm itself.
We happen to have the isotherm conveniently stored as a json file, so we load it into memory and we attempt to upload it.

In [8]:
import os
json_path = os.path.join(os.getcwd(), 'data', 'database', 'Carbon X1.json')
with open(json_path) as text_file:
    isotherm = pygaps.isotherm_from_json(text_file.read())

try:
    pygaps.db_upload_experiment(pygaps.DATABASE, isotherm)
except Exception as e:
    print(e)

Error on isotherm: 
 physisorption Carbon X1 Bob N2 homemade1 150.0 303.0 
 FOREIGN KEY constraint failed



As before, we need to first make sure that the constraints are satisfied before the isotherm can be uploaded. On the isotherm side, we have the following constraints:

- the adsorbate which was used in the experiment, which is already in the database
- the machine which was used to measure the data
- the user who measured it
- the experiment type, in this case physisorption
- the experiment data types, which in this case are just the columns in the dataframe: pressure and loading.


Again, we pass to the respective functions dictionaries with the required values.

In [9]:
pygaps.db_upload_machine(pygaps.DATABASE, {
                             'nick' : 'homemade1',
                             'name' : 'our own apparatus'
                         })

Machine uploaded homemade1


In [10]:
pygaps.db_upload_contact(pygaps.DATABASE, {
                             'nick' : 'Bob',
                             'name' : 'Dr. Bob'
                         })

Contact uploaded Bob


In [11]:
pygaps.db_upload_experiment_type(pygaps.DATABASE, {
                                 'type' : 'physisorption',
                                 'name' : 'physisorption at 77 k'
                             })

Experiment type uploaded physisorption


In [12]:
pygaps.db_upload_experiment_data_type(pygaps.DATABASE, {
                                 'type' : isotherm.loading_key,
                                 'name' : 'mmol/g'
                             })
pygaps.db_upload_experiment_data_type(pygaps.DATABASE, {
                                 'type' : isotherm.pressure_key,
                                 'unit' : 'bar'
                             })

Data type uploaded {'type': 'loading', 'name': 'mmol/g'}
Data type uploaded {'type': 'pressure', 'unit': 'bar'}


Now we can try the upload again.

In [13]:
pygaps.db_upload_experiment(pygaps.DATABASE, isotherm)

Success: Experimental isotherm
Material: Carbon
Batch: X1
Isotherm type: physisorption
Adsorbate used: N2
Machine: homemade1
User: Bob
Activation temperature: 150.0°C
Isotherm temperature: 303.0K



## Conclusion

While the instantiation process is initially lengthy, after the constraints are added to the database, the infrastructure is in place for easy upload of isotherms.

## Clean-up

If you run this iPython notebook with your own installation, run the following code to remove any residual values from the database.

In [14]:
pygaps.db_delete_experiment(pygaps.DATABASE, isotherm)
pygaps.db_delete_sample(pygaps.DATABASE, novel_sample)
pygaps.db_delete_source(pygaps.DATABASE, 'local')
pygaps.db_delete_contact(pygaps.DATABASE, 'Alice')
pygaps.db_delete_contact(pygaps.DATABASE, 'Bob')
pygaps.db_delete_machine(pygaps.DATABASE, 'homemade1')
pygaps.db_delete_sample_type(pygaps.DATABASE, 'novel')
pygaps.db_delete_sample_property_type(pygaps.DATABASE, 'chemical')
pygaps.db_delete_experiment_type(pygaps.DATABASE, 'physisorption')
pygaps.db_delete_experiment_data_type(pygaps.DATABASE, isotherm.loading_key)
pygaps.db_delete_experiment_data_type(pygaps.DATABASE, isotherm.pressure_key)

Success: Experimental isotherm
Material: Carbon
Batch: X1
Isotherm type: physisorption
Adsorbate used: N2
Machine: homemade1
User: Bob
Activation temperature: 150.0°C
Isotherm temperature: 303.0K

Success Carbon X1
Success local
Success Alice
Success Bob
Success homemade1
Success novel
Success chemical
Success physisorption
Success loading
Success pressure
